## Define modules

### Subtask:
Create separate Python files for each functional area (e.g., `file_utils.py`, `email_utils.py`, `scheduler.py`, `main.py`).


**Reasoning**:
Create the empty Python files as instructed.



In [15]:
!touch file_utils.py email_utils.py scheduler.py main.py

## Implement file utilities

### Subtask:
Write the code for file operations (copy, move, delete) in `file_utils.py`.


In [ ]:
%%writefile file_utils.py
import os
import shutil

def copy_file(source, destination):
    """Copies a file from source to destination."""
    try:
        shutil.copy2(source, destination)
        print(f"Successfully copied '{source}' to '{destination}'")
    except FileNotFoundError:
        print(f"Error: Source file not found at '{source}'")
    except PermissionError:
        print(f"Error: Permission denied to copy '{source}' to '{destination}'")
    except Exception as e:
        print(f"An unexpected error occurred while copying '{source}': {e}")

def move_file(source, destination):
    """Moves a file from source to destination."""
    try:
        shutil.move(source, destination)
        print(f"Successfully moved '{source}' to '{destination}'")
    except FileNotFoundError:
        print(f"Error: Source file not found at '{source}'")
    except PermissionError:
        print(f"Error: Permission denied to move '{source}' to '{destination}'")
    except Exception as e:
        print(f"An unexpected error occurred while moving '{source}': {e}")

def delete_file(filepath):
    """Deletes a file."""
    try:
        os.remove(filepath)
        print(f"Successfully deleted '{filepath}'")
    except FileNotFoundError:
        print(f"Error: File not found at '{filepath}'")
    except PermissionError:
        print(f"Error: Permission denied to delete '{filepath}'")
    except Exception as e:
        print(f"An unexpected error occurred while deleting '{filepath}': {e}")

## Implement email utilities

### Subtask:
Write the code for sending emails in `email_utils.py`. Note that this will be initially commented out and require user configuration.


In [ ]:
%%writefile email_utils.py
# import smtplib
# from email.mime.text import MIMEText
#
# def send_email(subject, body, to_email):
#     """Sends an email using SMTP."""
#     # --- User Configuration ---
#     # Replace with your email address
#     from_email = "your_email@example.com"
#     # Replace with your email password or app-specific password
#     password = "your_password"
#     # Replace with your SMTP server details
#     smtp_server = "smtp.example.com"
#     smtp_port = 587
#     # --------------------------
#
#     msg = MIMEText(body)
#     msg['Subject'] = subject
#     msg['From'] = from_email
#     msg['To'] = to_email
#
#     try:
#         with smtplib.SMTP(smtp_server, smtp_port) as server:
#             server.starttls()
#             server.login(from_email, password)
#             server.sendmail(from_email, [to_email], msg.as_string())
#             print(f"Successfully sent email to '{to_email}'")
#     except Exception as e:
#         print(f"An error occurred while sending the email: {e}")


## Implement scheduler

### Subtask:
Write the code for scheduling and running tasks in `scheduler.py`.


In [ ]:
%%writefile scheduler.py
import schedule
import time
import threading

stop_scheduler = False

def schedule_task(task_func, schedule_time):
    """Schedules a task function at a specific time."""
    try:
        # Assuming schedule_time is in a format like "HH:MM"
        schedule.every().day.at(schedule_time).do(task_func)
        print(f"Task scheduled daily at {schedule_time}")
    except Exception as e:
        print(f"Error scheduling task: {e}")

def run_scheduled_tasks():
    """Continuously checks for and runs pending scheduled tasks."""
    global stop_scheduler
    while not stop_scheduler:
        schedule.run_pending()
        time.sleep(1)

def start_scheduler_thread():
    """Starts the scheduler in a separate thread."""
    scheduler_thread = threading.Thread(target=run_scheduled_tasks)
    scheduler_thread.daemon = True  # Allows the main program to exit even if the thread is running
    scheduler_thread.start()
    print("Scheduler started.")

def stop_scheduler_thread():
    """Stops the scheduler thread."""
    global stop_scheduler
    stop_scheduler = True
    print("Scheduler stopping...")

if __name__ == '__main__':
    # Example Usage (for testing the scheduler module directly)
    def my_task():
        print("Task executed!")

    # Schedule the task to run every day at a specific time (e.g., 10:30 AM)
    # schedule_task(my_task, "10:30")

    # Or schedule it to run in 5 seconds for testing
    schedule.every(5).seconds.do(my_task)
    print("Task scheduled to run in 5 seconds.")


    start_scheduler_thread()

    # Keep the main thread alive to allow the scheduler thread to run
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        stop_scheduler_thread()
        print("Scheduler stopped by user.")


## Create main application

### Subtask:
Write the main script (`main.py`) that provides the command-line interface and integrates the other modules.


In [ ]:
%%writefile main.py
import argparse
import time
import sys

# Assume file_utils.py and scheduler.py are in the same directory
import file_utils
import scheduler

def main():
    """Main function to parse arguments and execute tasks."""
    parser = argparse.ArgumentParser(description="Personal project for file operations and scheduling.")

    # Mutually exclusive group for file operations
    file_ops_group = parser.add_mutually_exclusive_group()

    file_ops_group.add_argument(
        "--copy",
        nargs=2,
        metavar=('SOURCE', 'DESTINATION'),
        help="Copy a file from SOURCE to DESTINATION."
    )
    file_ops_group.add_argument(
        "--move",
        nargs=2,
        metavar=('SOURCE', 'DESTINATION'),
        help="Move a file from SOURCE to DESTINATION."
    )
    file_ops_group.add_argument(
        "--delete",
        metavar='FILEPATH',
        help="Delete a file at FILEPATH."
    )

    # Arguments for scheduling tasks
    parser.add_argument(
        "--schedule-task",
        nargs='+',
        metavar=('TASK_TYPE', 'SCHEDULE_TIME', 'ARGS...'),
        help="Schedule a task. TASK_TYPE can be 'copy', 'move', or 'delete'. SCHEDULE_TIME is in HH:MM format. ARGS are task-specific."
    )

    args = parser.parse_args()

    # Handle file operations
    if args.copy:
        source, destination = args.copy
        file_utils.copy_file(source, destination)
    elif args.move:
        source, destination = args.move
        file_utils.move_file(source, destination)
    elif args.delete:
        filepath = args.delete
        file_utils.delete_file(filepath)
    elif args.schedule_task:
        task_type = args.schedule_task[0].lower()
        if len(args.schedule_task) < 2:
            print("Error: SCHEDULE_TIME is required for scheduling tasks.")
            sys.exit(1)
        schedule_time = args.schedule_task[1]
        task_args = args.schedule_task[2:]

        # Define a helper function to run the scheduled task
        def scheduled_file_task():
            if task_type == 'copy' and len(task_args) == 2:
                source, destination = task_args
                file_utils.copy_file(source, destination)
            elif task_type == 'move' and len(task_args) == 2:
                source, destination = task_args
                file_utils.move_file(source, destination)
            elif task_type == 'delete' and len(task_args) == 1:
                filepath = task_args[0]
                file_utils.delete_file(filepath)
            else:
                print(f"Error: Invalid task type or arguments for scheduled task: {task_type} with args {task_args}")

        scheduler.schedule_task(scheduled_file_task, schedule_time)
        scheduler.start_scheduler_thread()

        # Keep the main thread alive to allow the scheduler thread to run
        try:
            print("Scheduler running. Press Ctrl+C to stop.")
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            scheduler.stop_scheduler_thread()
            print("Main program stopping.")
            sys.exit(0)

    else:
        # If no arguments are provided, display help message
        parser.print_help()

if __name__ == "__main__":
    main()